# **Final Project: Khai thác và trực quan hóa dữ liệu**

### **Danh sách các thành viên**

<center>

|         Họ tên         |   MSSV   |
| :--------------------- | :------: |
| Trần Nguyễn Nhật Cường | 22127048 |
|    Nguyễn Công Tuấn    | 22127436 |
|     Trần Đăng Tuấn     | 22127438 |
| ... | ... |
| ... | ... |

</center>

## **Cài đặt và sử dụng các thư viện cần thiết**

In [ ]:
import numpy as np
import pandas as pd
import requests

**`matplotlib`** được dùng để vẽ biểu đồ trong Python. Có thể dùng để vẽ các loại biểu đồ như
histogram, biểu đồ tán xạ (scatter plot), biểu đồ đường (line chart) và biểu đồ cột (bar chart),...

**`pandas`** được dùng để xử lý và phân tích dữ liệu dạng bảng và cung cấp cấu trúc dữ liệu như
DataFrame và Series để dễ thao tác.

**`seaborn`** dùng để vẽ biểu đồ dựa trên thư viện matplotlib nhưng biểu đồ sẽ trông đẹp hơn và cũng hỗ trợ các loại biểu đồ thống kê như violin plot, box plot, heatmap.

**`numpy`** được dùng để xử lý các phép toán số học nhanh trên mảng nhiều chiều (arrays) và hỗ trợ các phép toán ma trận, thống kê, đại số tuyến tính cần thiết cho phân tích dữ liệu.


**`requests`** được dùng để gửi các yêu cầu HTTP (như GET, POST) nhằm truy vấn dữ liệu từ các API hoặc tải dữ liệu từ Internet về máy tính.

## **Thu thập dữ liệu**

In [ ]:
offset = 0
limit = 1000
all_records = []

while True:
    response = requests.get(
        url='https://data.vietnam.opendevelopmentmekong.net/en/api/3/action/datastore_search',
        params={
            'resource_id': 'a03fb5c6-4622-4196-a926-15e9542c4ae8',
            'limit': limit,
            'offset': offset
        }
    )
    
    records = response.json()['result']['records']
    if not records:
        break
    all_records.extend(records)
    offset += limit

df = pd.DataFrame(all_records)
df.head()

## **Khám phá dữ liệu**

<div align='center'>

| Cột | Mô tả | 
| --- | --- |
| `_id` | 	ID tự động của hệ thống để định danh từng hàng trong cơ sở dữ liệu |
| `ID` | Mã số định danh riêng của từng khu đất/lô đất |
| `Latitude` | 	Vĩ độ (tọa độ địa lý) của khu đất/lô đất |
| `Longtitude` | Kinh độ (tọa độ địa lý) của khu đất/lô đất. |
| `Province` | Tên tỉnh/thành phố nơi khu đất/lô đất tọa lạc. |
| `District` | Tên quận/huyện nơi khu đất/lô đất tọa lạc. |
| `Commune` | Tên xã/phường/thị trấn nơi khu đất/lô đất tọa lạc. |
| `Sub-regional` | 	Khu vực/tiểu vùng nhỏ hơn trong xã hoặc huyện (thường là cụm nhỏ hơn). |
| `Plot` | Số hiệu thửa đất (Plot number). |
| `Lot` | Số hiệu lô đất trong thửa đất. |
| `Site` | Tên cụ thể của khu vực/quần thể đất (ví dụ: tên thôn, làng). |
| `Forest owner	` | Tên chủ sở hữu hoặc đơn vị quản lý rừng. |
| `Land type` | Loại hình đất tại khu vực (ví dụ: rừng tự nhiên trung bình, đất rừng trồng...). |
| `Original resources of forest` | Loại tài nguyên rừng ban đầu (ví dụ: rừng tự nhiên, rừng trồng...). |
| `Date` | Ngày ghi nhận hoặc cập nhật thông tin khu đất/lô đất. |
| `Priority` | Mức độ ưu tiên quản lý hoặc bảo vệ khu đất/lô đất (ví dụ: High - Cao). | 
| `Contractor` | Tên nhà thầu hoặc cá nhân phụ trách khai thác hoặc trồng rừng (nếu có). |
| `Status` | Tình trạng hiện tại của khu đất/lô đất (ví dụ: Verified - Đã xác minh). |

</div>

In [ ]:
df.info()

## **Tiền xử lý dữ liệu**

In [ ]:
df.replace(to_replace='', value=np.nan, inplace=True)
round(df.isna().sum() / df.shape[0] * 100, 2)

In [ ]:
df.drop(columns=['_id', 'ID', 'Contractor'], inplace=True)
df.head()

In [ ]:
df.rename(columns={
    'Longtitude': 'Longitude'
}, inplace=True)

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)
df.duplicated().sum()

In [ ]:
df['Date'] = pd.to_datetime(arg=df['Date'], format='%Y-%m-%dT%H:%M:%S')
df['Date'].head()

In [ ]:
for col in df.select_dtypes(include='object').columns:
    print(f'{col}: {df[col].unique()}')

**Nhận xét**
- `Nam Can` có thể là xã **Nậm Càn** hoặc **Nậm Cắn**, cần dựa vào vĩ độ và kinh độ để xác định chính xác đó là xã nào.

In [ ]:
df[df['Commune'] == 'Nam Can']

**Nhận xét**
- Dựa vào **vĩ độ** và **kinh độ**, nhóm xác định được cả 3 vị trí này đều thuộc xã **Nam Càn**, huyện **Kỳ Sơn**, tỉnh **Nghệ An**.

**Tổng kết các tỉnh/huyện/xã**
- **Lâm Đồng (20)**:
    - **Di Linh (14)**: Bảo Thuận, Đinh Lạc, Đinh Trang Thượng, Gia Bắc, Gia Hiệp, Gung Ré, Hòa Bắc, Hòa Trung, Liên Đầm, Sơn Điền, Tam Bố, Tân Châu, Tân Lâm, Tân Thượng.
    - **Lạc Dương (6)**: Thị trấn Lạc Dương, Đạ Chais, Đạ Nhim, Đạ Sar, Đưng KNớ, Lát.

- **Nghệ An (15)**:
    - **Tương Dương (6)**: Thị trấn Thạch Giám, Tam Đinh, Tam Hợp, Tam Quang, Tam Thái, Yên Thắng.
    - **Kỳ Sơn (9)**: Chiêu Lưu, Hữu Kiệm, Hữu Lập, Mường Típ, Na Ngoi, Nậm Càn, Phà Đánh, Tà Cạ, Tây Sơn.

In [ ]:
df['Province'] = df['Province'].replace(to_replace={
    'Lam Dong': 'Lâm Đồng',
    'Nghe An': 'Nghệ An'
})

df['District'] = df['District'].replace(to_replace={
    # tỉnh Lâm Đồng
    'Lac Duong': 'Lạc Dương',
    # tỉnh Nghệ An
    'Tuong Duong': 'Tương Dương',
    'Ky Son': 'Kỳ Sơn'
})

df['Commune'] = df['Commune'].replace(to_replace={
    # huyện Di Linh
    'Bao Thuan': 'Bảo Thuận', 'Đinh Lạc': 'Đinh Lạc',
    'Dinh Trang Thuong': 'Đinh Trang Thượng',
    'Gia Bac': 'Gia Bắc', 'Gia Hiep': 'Gia Hiệp',
    'Gung Re': 'Gung Ré', 'Hoa Bac': 'Hòa Bắc',
    'Hoa Trung': 'Hòa Trung', 'Lien Dam': 'Liên Đầm',
    'Son Dien': 'Sơn Điền', 'Tam Bo': 'Tam Bố',
    'Tan Chau': 'Tân Châu', 'Tan Lam': 'Tân Lâm',
    'Tan Thuong': 'Tân Thượng',
    # huyện Lạc Dương
    'Lac Duong Town': 'Lạc Dương', 'Da Chais': 'Đạ Chais',
    'Da Sar': 'Đạ Sar', 'Dung KN\'o': 'Đưng KNớ',
    'Lat': 'Lát',
    # huyện Tương Dương
    'Thach Giam': 'Thạch Giám', 'Tam Dinh': 'Tam Đinh',
    'Tam Hop': 'Tam Hợp', 'Tam Thai': 'Tam Thái',
    'Yen Thang': 'Yên Thắng',
    # huyện Kỳ Sơn
    'Chieu Luu': 'Chiêu Lưu', 'Huu Kiem': 'Hữu Kiệm',
    'Huu Lap': 'Hữu Lập', 'Muong Tip': 'Mường Típ',
    'Nam Can': 'Nậm Càn', 'Pha Danh': 'Phà Đánh',
    'Ta Ca': 'Tà Cạ', 'Tay Son': 'Tây Sơn'
})

**Forest owner**

<center>

|             <center> Tên tiếng Anh </center>              |          <center> Tên tiếng Việt </center>          |
|:----------------------------------------------------------|:----------------------------------------------------|
| Forestry One Member Limited Liability Company Di Linh     | Công ty TNHH một thành viên lâm nghiệp Di Linh.     |
| Board of protection forest management Tan Thuong          | Ban Quản lý rừng Tân Thượng.                        |
| Group Joint Stock Company Tan Mai                         | Công ty cổ phần Tập đoàn Tân Mai.                   |
| Community of  Da Hong village                             | Cộng đồng thôn Đà Hòng.                             |
| Forestry One Member Limited Liability Company Bao Thuan   | Công ty TNHH một thành viên lâm nghiệp Bảo Thuận.   |
| Commune People's Committee                                | Ủy ban nhân dân xã.                                 |
| Forestry One Member Limited Liability Company Tam Hiep    | Công ty TNHH một thành viên lâm nghiệp Tam Hiệp.    |
| Private enterprise Tan Minh                               | Doanh Nghiệp Tư Nhân Tân Minh.                      |
| BQL RPH Hũa Bắc - Hũa Nam                                 | Ban Quản lý Rừng phòng hộ Hòa Bắc - Hòa Nam.        |
| Forestry One Member Limited Liability Company Phat Lam    | -                                                   |
| Limited Liability Company Tien Le                         | Công ty TNHH xuất nhập khẩu Tiến Lê.                |
| Private enterprise Le Tam                                 | -                                                   |
| Limited Liability Company Vinh Bao                        | Công ty TNHH Vĩnh Bảo.                              |
| National park Pu Mat                                      | Vườn quốc gia Pù Mát.                               |
| Board of protection forest management Tuong Duong         | Ban Quản lý rừng phòng hộ Tương Dương.              |
| Forestry One Member Limited Liability Company Tuong Duong | Công ty TNHH một thành viên lâm nghiệp Tương Dương. |
| HGD                                                       | Hộ gia đình                                         |
| Limited Liability Company Lam Sinh Tai                    | Công ty TNHH Lâm Sinh Tài                           |
| Community of  village 6                                   | Cộng đồng thôn 6                                    |
| Youth Volunteer Team                                      | Đội thanh niên tình nguyện                          |
| Board of protection forest management Ky Son              | Ban Quản lý rừng phòng hộ Kỳ Sơn                    |

</center>

In [ ]:
df['Forest owner'] = df['Forest owner'].replace(to_replace={
    'Forestry One Member Limited Liability Company Di Linh': 'Công ty TNHH một thành viên lâm nghiệp Di Linh',
    'Board of protection forest management Tan Thuong': 'Ban Quản lý rừng Tân Thượng',
    'Group Joint Stock Company Tan Mai': 'Công ty cổ phần Tập đoàn Tân Mai',
    'Community of  Da Hong village': 'Cộng đồng thôn Đà Hòng',
    'Forestry One Member Limited Liability Company Bao Thuan': 'Công ty TNHH một thành viên lâm nghiệp Bảo Thuận',
    'Commune People\'s Committee': 'Ủy ban nhân dân xã',
    'Forestry One Member Limited Liability Company Tam Hiep': 'Công ty TNHH một thành viên lâm nghiệp Tam Hiệp',
    'Private enterprise Tan Minh': 'Doanh Nghiệp Tư Nhân Tân Minh',
    'BQL RPH Hũa Bắc - Hũa Nam': 'Ban Quản lý Rừng phòng hộ Hòa Bắc - Hòa Nam',
    'Forestry One Member Limited Liability Company Phat Lam': 'Không rõ',
    'Limited Liability Company Tien Le': 'Công ty TNHH xuất nhập khẩu Tiến Lê',
    'Private enterprise Le Tam': 'Không rõ',
    'Limited Liability Company Vinh Bao': 'Công ty TNHH Vĩnh Bảo',
    'National park Pu Mat': 'Vườn quốc gia Pù Mát',
    'Board of protection forest management Tuong Duong': 'Ban Quản lý rừng phòng hộ Tương Dương',
    'Forestry One Member Limited Liability Company Tuong Duong': 'Công ty TNHH một thành viên lâm nghiệp Tương Dương',
    'HGD': 'Hộ gia đình',
    'Limited Liability Company Lam Sinh Tai': 'Công ty TNHH Lâm Sinh Tài',
    'Community of  village 6': 'Cộng đồng thôn 6',
    'Board of protection forest management Ky Son': 'Ban Quản lý rừng phòng hộ Kỳ Sơn',
    np.nan: 'Không rõ'
})
df['Forest owner'] = df['Forest owner'].str.replace(pat='Youth Volunteer Team', repl='Đội thanh niên tình nguyện')

df['Forest owner'].unique()

In [ ]:
df['Land type'].unique()